In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructField,StringType,IntegerType,DoubleType,StructType,TimestampType)
from pyspark.sql.functions import *
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('SparkByCredit').getOrCreate()

In [3]:
df = pd.read_json("../../dataset/creditcard_dataset/cdw_sapp_credit.json",lines=True)

In [4]:
df.head()

,TRANSACTION_ID,DAY,MONTH,YEAR,CREDIT_CARD_NO,CUST_SSN,BRANCH_CODE,TRANSACTION_TYPE,TRANSACTION_VALUE
0,1,14,2,2018,4210653349028689,123459988,114,Education,78.90
1,2,20,3,2018,4210653349028689,123459988,35,Entertainment,14.24
2,3,8,7,2018,4210653349028689,123459988,160,Grocery,56.70
3,4,19,4,2018,4210653349028689,123459988,114,Entertainment,59.73
4,5,10,10,2018,4210653349028689,123459988,93,Gas,3.59


In [16]:
df.dtypes

TRANSACTION_ID         int64
DAY                    int64
MONTH                  int64
YEAR                   int64
CREDIT_CARD_NO         int64
CUST_SSN               int64
BRANCH_CODE            int64
TRANSACTION_TYPE      object
TRANSACTION_VALUE    float64
dtype: object

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46694 entries, 0 to 46693
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TRANSACTION_ID     46694 non-null  int64  
 1   DAY                46694 non-null  int64  
 2   MONTH              46694 non-null  int64  
 3   YEAR               46694 non-null  int64  
 4   CREDIT_CARD_NO     46694 non-null  int64  
 5   CUST_SSN           46694 non-null  int64  
 6   BRANCH_CODE        46694 non-null  int64  
 7   TRANSACTION_TYPE   46694 non-null  object 
 8   TRANSACTION_VALUE  46694 non-null  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 3.2+ MB


In [5]:

df = df.rename(columns={"CREDIT_CARD_NO":"CUST_CC_NO"})
df["TRANSACTION_TYPE"] = df["TRANSACTION_TYPE"].astype(str)
df["TRANSACTION_VALUE"] = df["TRANSACTION_VALUE"].astype(float)
df["CUST_CC_NO"] = df["CUST_CC_NO"].astype(str)
df['date'] = pd.to_datetime(df[['YEAR','MONTH','DAY']], format='%d-%m-%Y')
df['TIMEID'] = df['date'].dt.strftime('%Y%m%d')
df = df.drop(['DAY',"MONTH","YEAR"], axis=1)
df = df.drop("date",axis=1)
df




,TRANSACTION_ID,CUST_CC_NO,CUST_SSN,BRANCH_CODE,TRANSACTION_TYPE,TRANSACTION_VALUE,TIMEID
0,1,4210653349028689,123459988,114,Education,78.90,20180214
1,2,4210653349028689,123459988,35,Entertainment,14.24,20180320
2,3,4210653349028689,123459988,160,Grocery,56.70,20180708
3,4,4210653349028689,123459988,114,Entertainment,59.73,20180419
4,5,4210653349028689,123459988,93,Gas,3.59,20181010
...,...,...,...,...,...,...,...
46689,46690,4210653344660822,123451007,49,Gas,66.20,20180912
46690,46691,4210653344660822,123451007,168,Grocery,100.13,20180205
46691,46692,4210653344660822,123451007,104,Test,35.83,20181216
46692,46693,4210653344660822,123451007,32,Entertainment,90.99,20180115


In [26]:
spark_df = spark.createDataFrame(df)

C:\Spark\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [28]:
spark_df.show()

+--------------+----------------+---------+-----------+----------------+-----------------+--------+
|TRANSACTION_ID|      CUST_CC_NO| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+--------------+----------------+---------+-----------+----------------+-----------------+--------+
|             1|4210653349028689|123459988|        114|       Education|             78.9|20180214|
|             2|4210653349028689|123459988|         35|   Entertainment|            14.24|20180320|
|             3|4210653349028689|123459988|        160|         Grocery|             56.7|20180708|
|             4|4210653349028689|123459988|        114|   Entertainment|            59.73|20180419|
|             5|4210653349028689|123459988|         93|             Gas|             3.59|20181010|
|             6|4210653349028689|123459988|        164|       Education|             6.89|20180528|
|             7|4210653349028689|123459988|        119|   Entertainment|            43.39|20180519|


In [ ]:
host_name = 'database-XXXX.us-west-1.rds.amazonaws.com'
dbname = 'XXXX'
port = '5432'
username = 'XXXX'
password = 'XXXXX'

In [ ]:
def connect_to_db(host_name, dbname, port, username, password):
   try:
       conn = ps.connect(host=host_name, database=dbname, user=username,
       password=password, port=port)
 
   except ps.OperationalError as e:
       raise e
   else:
       print('Connected!')
       return conn